In [23]:
pip install pyspark

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, regexp_replace, avg, max, min, count
from pyspark.sql.functions import round


# Initialize Spark session
spark = SparkSession.builder \
    .appName("Quikr Car Dataset Analysis") \
    .getOrCreate()

# Load the dataset
df = spark.read.csv("quikr_car.csv", header=True, inferSchema=True)

# Display schema and sample data
df.printSchema()
df.show(5)

root
 |-- name: string (nullable = true)
 |-- company: string (nullable = true)
 |-- year: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- kms_driven: string (nullable = true)
 |-- fuel_type: string (nullable = true)

+--------------------+--------+----+-------------+----------+---------+
|                name| company|year|        Price|kms_driven|fuel_type|
+--------------------+--------+----+-------------+----------+---------+
|Hyundai Santro Xi...| Hyundai|2007|       80,000|45,000 kms|   Petrol|
|Mahindra Jeep CL5...|Mahindra|2006|     4,25,000|    40 kms|   Diesel|
|Maruti Suzuki Alt...|  Maruti|2018|Ask For Price|22,000 kms|   Petrol|
|Hyundai Grand i10...| Hyundai|2014|     3,25,000|28,000 kms|   Petrol|
|Ford EcoSport Tit...|    Ford|2014|     5,75,000|36,000 kms|   Diesel|
+--------------------+--------+----+-------------+----------+---------+
only showing top 5 rows



In [7]:
#Step-2: Data Cleaning

# Clean 'Price' column: remove non-numeric characters and convert to integer
df = df.withColumn("Price", regexp_replace(col("Price"), "[^0-9]", ""))
df = df.withColumn("Price", col("Price").cast("int"))

# Clean 'kms_driven' column: remove non-numeric characters and convert to integer
df = df.withColumn("kms_driven", regexp_replace(col("kms_driven"), "[^0-9]", ""))
df = df.withColumn("kms_driven", col("kms_driven").cast("int"))

# Drop rows with nulls in important columns
df_clean = df.dropna(subset=["Price", "kms_driven", "year", "fuel_type", "company"])
print('Data Cleaned')

Data Cleaned


In [9]:
#Analysis
# 1. Average Price by Company
avg_price_by_company = df_clean.groupBy("company") \
    .agg(round(avg("Price"), 2).alias("avg_price"))

avg_price_by_company.show()


+----------+----------+
|   company| avg_price|
+----------+----------+
|    Maruti| 269837.55|
|Volkswagen|  407105.0|
|    Jaguar| 2495000.0|
|      Tata| 200939.29|
|      Jeep|  950000.0|
|Mitsubishi|1298333.33|
|      Mini| 1891111.0|
| Chevrolet| 202085.26|
|     Volvo| 1850000.0|
|   Hyundai| 318402.78|
|     Honda| 347632.92|
| Hindustan| 303333.33|
|      Audi|1476909.09|
|  Mercedes|1170666.83|
|      Land| 2100000.0|
|   Renault| 460512.27|
|  Mahindra| 603436.69|
|       BMW| 1049875.0|
|    Datsun|  255714.0|
|     Skoda| 251191.38|
+----------+----------+
only showing top 20 rows



In [11]:
# 2. Average Kilometers Driven by Fuel Type
avg_kms_by_fuel = df_clean.groupBy("fuel_type") \
    .agg(round(avg("kms_driven"), 2).alias("avg_kms_driven"))

avg_kms_by_fuel.show()

+---------+--------------+
|fuel_type|avg_kms_driven|
+---------+--------------+
|   Diesel|      55650.98|
|      LPG|      115000.0|
|   Petrol|      37498.96|
+---------+--------------+



In [13]:
# 3. Count of Cars by Year
car_count_by_year = df_clean.groupBy("year").agg(count("*").alias("car_count"))
car_count_by_year.orderBy("year").show()

+----+---------+
|year|car_count|
+----+---------+
|1995|        2|
|2000|        7|
|2001|        5|
|2002|        4|
|2003|       13|
|2004|       12|
|2005|       13|
|2006|       22|
|2007|       19|
|2008|       16|
|2009|       54|
|2010|       43|
|2011|       59|
|2012|       75|
|2013|       94|
|2014|       92|
|2015|      111|
|2016|       74|
|2017|       53|
|2018|       30|
+----+---------+
only showing top 20 rows



In [15]:
# 4. Max/Min Price by Company
price_stats_by_company = df_clean.groupBy("company").agg(
    max("Price").alias("max_price"),
    min("Price").alias("min_price")
)
price_stats_by_company.show()


+----------+---------+---------+
|   company|max_price|min_price|
+----------+---------+---------+
|    Maruti|  1000000|    30000|
|Volkswagen|  1200000|   199999|
|    Jaguar|  2800000|  2190000|
|      Tata|   650000|    32000|
|      Jeep|   950000|   950000|
|Mitsubishi|  1725000|    85000|
|      Mini|  1891111|  1891111|
| Chevrolet|   490000|    80000|
|     Volvo|  1850000|  1850000|
|   Hyundai|  1200000|    45000|
|     Honda|   860000|    65000|
| Hindustan|   750000|    70000|
|      Audi|  3100000|   500000|
|  Mercedes|  2000000|   399000|
|      Land|  2100000|  2100000|
|   Renault|   800000|   244999|
|  Mahindra|  8500003|   130000|
|       BMW|  1600000|   600000|
|    Datsun|   299999|   235000|
|     Skoda|   530000|   110000|
+----------+---------+---------+
only showing top 20 rows



**Conclusion**:In this task, we used PySpark to perform big data analysis on the `quikr_car.csv` dataset. We cleaned and transformed key columns like `Price` and `kms_driven`, then applied grouped operations to derive insights such as average price by company, average kilometers driven by fuel type, and car distribution by year. This demonstrated PySpark’s scalability and efficiency in processing large datasets, fulfilling the requirement to extract insights using big data tools.
